In [1]:
import pandas as pd
import os
import glob
import csv

### File aquiration template

In [25]:
# Source file cleaning
# Fill in the source number
source_number = 2
# Fill in your own file path, have the folder containing the csv files be called "Source_{number}"
# Make sure to change any \ to / to avoid errors
source_folder_path = f"data/Source_{source_number}"

# Open the source folder to take all CSV's from
csv_files_Source = glob.glob(os.path.join(f"{source_folder_path}", "*.csv"))

# Open the file to store the new CSV in
compiled_file_source = open(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv", "a")
# Start the CSV writer
compiled_file_source_writer = csv.writer(compiled_file_source)

# Go through all raw source files in the provided folder
for file_path in csv_files_Source:
    raw_file = open(file_path, "r")
    # Split the input file in lines
    raw_file_lines = raw_file.readlines()
    for line in raw_file_lines:
        # Split each line on the column split symbol ";"
        raw_line_split = line.split(';')
        # Add the split row to the new file
        compiled_file_source_writer.writerow(raw_line_split)
    raw_file.close()
compiled_file_source.close()

# Pandas sanity check, open the just created file
pd.read_csv(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv")

,"﻿""Perioden""","""Regio's""","""Bevolking/Bevolkingssamenstelling op 1 januari/Totale bevolking (aantal)""","""Bevolking/Bevolkingssamenstelling op 1 januari/Geslacht/Mannen (aantal)""","""Bevolking/Bevolkingssamenstelling op 1 januari/Geslacht/Vrouwen (aantal)""","""Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/Jonger dan 5 jaar (aantal)""","""Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/5 tot 10 jaar (aantal)""","""Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/10 tot 15 jaar (aantal)""","""Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/15 tot 20 jaar (aantal)""","""Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/20 tot 25 jaar (aantal)""",...,"""Onderwijs/Naar woongemeente/Gediplomeerden/Wo master/doctoraal (aantal)""","""Arbeid/Banen van werknemers, relatief/A Landbouw, bosbouw en visserij (%)""","""Arbeid/Banen van werknemers, relatief/B-F Nijverheid en energie (%)""","""Arbeid/Banen van werknemers, relatief/G-N Commerciële dienstverlening (%)""","""Arbeid/Banen van werknemers, relatief/O-U Niet-commerciële dienstverlening (%)""","""Nabijheid voorzieningen/Gezondheid/Afstand tot huisartsenpraktijk (km)""","""Nabijheid voorzieningen/Gezondheid/Aantal huisartsenpraktijken binnen 3 km (aantal)""","""Nabijheid voorzieningen/Gezondheid/Afstand tot huisartsenpost (km)""","""Nabijheid voorzieningen/Gezondheid/Afstand tot ziekenhuis (km)""","""Nabijheid voorzieningen/Gezondheid/Aantal ziekenhuizen binnen 20 km (aantal)""\r\n"
0,"""2019""","""Aa en Hunze""",25386.0,12659.0,12727.0,922.0,1174.0,1366.0,1579.0,1031.0,...,8.0,1.0,10.0,45.0,44.0,"2,4","1,3","11,1","11,1","1,4\r\n"
1,"""2020""","""Aa en Hunze""",25445.0,12708.0,12737.0,946.0,1160.0,1354.0,1545.0,1079.0,...,NaN,1.0,8.0,43.0,47.0,"2,4","1,1","11,1","11,1","1,4\r\n"
2,"""2021""","""Aa en Hunze""",25399.0,12709.0,12690.0,937.0,1137.0,1309.0,1508.0,1103.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\r\n
3,"""2019""","""Aalburg""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\r\n
4,"""2020""","""Aalburg""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\r\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8422,"""\r\n",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8423,\r\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8424,"""""""2021"""""",""""""De Wolden"""""",24374,12197,12177,1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8425,"""\r\n",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Source 1 preprocessing

In [3]:
# INFO ONLY ON 2021

In [24]:
# Source file cleaning
source_number = 1
source_folder_path = f"data/Source_{source_number}"
# Open the source folder to take all CSV's from
csv_files_Source = glob.glob(os.path.join(f"{source_folder_path}", "*.csv"))

# Open the file to store the new CSV in
compiled_file_source = open(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv", "a")
# Start the CSV writer
compiled_file_source_writer = csv.writer(compiled_file_source)

# Go through all raw source files in the provided folder
for file_path in csv_files_Source:
    raw_file = open(file_path, "r")
    # Split the input file in lines
    raw_file_lines = raw_file.read().splitlines()
    for line in raw_file_lines:
        # Split each line on the column split symbol ";"
        raw_line_split = line.split(';')
        # Add the split row to the new file
        compiled_file_source_writer.writerow(raw_line_split)
    raw_file.close()
compiled_file_source.close()

In [26]:
# Pandas sanity check, open the just created file
pd.read_csv(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv")

,"﻿""Perioden""","""Regio's""","""Bevolking/Bevolkingssamenstelling op 1 januari/Totale bevolking (aantal)""","""Bevolking/Bevolkingssamenstelling op 1 januari/Geslacht/Mannen (aantal)""","""Bevolking/Bevolkingssamenstelling op 1 januari/Geslacht/Vrouwen (aantal)""","""Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/Jonger dan 5 jaar (aantal)""","""Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/5 tot 10 jaar (aantal)""","""Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/10 tot 15 jaar (aantal)""","""Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/15 tot 20 jaar (aantal)""","""Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/20 tot 25 jaar (aantal)""",...,"""Onderwijs/Naar woongemeente/Gediplomeerden/Wo master/doctoraal (aantal)""","""Arbeid/Banen van werknemers, relatief/A Landbouw, bosbouw en visserij (%)""","""Arbeid/Banen van werknemers, relatief/B-F Nijverheid en energie (%)""","""Arbeid/Banen van werknemers, relatief/G-N Commerciële dienstverlening (%)""","""Arbeid/Banen van werknemers, relatief/O-U Niet-commerciële dienstverlening (%)""","""Nabijheid voorzieningen/Gezondheid/Afstand tot huisartsenpraktijk (km)""","""Nabijheid voorzieningen/Gezondheid/Aantal huisartsenpraktijken binnen 3 km (aantal)""","""Nabijheid voorzieningen/Gezondheid/Afstand tot huisartsenpost (km)""","""Nabijheid voorzieningen/Gezondheid/Afstand tot ziekenhuis (km)""","""Nabijheid voorzieningen/Gezondheid/Aantal ziekenhuizen binnen 20 km (aantal)""\r\n"
0,"""2019""","""Aa en Hunze""",25386.0,12659.0,12727.0,922.0,1174.0,1366.0,1579.0,1031.0,...,8.0,1.0,10.0,45.0,44.0,"2,4","1,3","11,1","11,1","1,4\r\n"
1,"""2020""","""Aa en Hunze""",25445.0,12708.0,12737.0,946.0,1160.0,1354.0,1545.0,1079.0,...,NaN,1.0,8.0,43.0,47.0,"2,4","1,1","11,1","11,1","1,4\r\n"
2,"""2021""","""Aa en Hunze""",25399.0,12709.0,12690.0,937.0,1137.0,1309.0,1508.0,1103.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\r\n
3,"""2019""","""Aalburg""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\r\n
4,"""2020""","""Aalburg""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\r\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8422,"""\r\n",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8423,\r\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8424,"""""""2021"""""",""""""De Wolden"""""",24374,12197,12177,1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8425,"""\r\n",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Source 2 preprocessing

In [6]:
# Source file cleaning
source_number = 2
source_folder_path = f"data/Source_{source_number}"
# Open the source folder to take all CSV's from
csv_files_Source = glob.glob(os.path.join(f"{source_folder_path}", "*.csv"))

# Open the file to store the new CSV in
compiled_file_source = open(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv", "a")
# Start the CSV writer
compiled_file_source_writer = csv.writer(compiled_file_source)

# Go through all raw source files in the provided folder
for file_path in csv_files_Source:
    raw_file = open(file_path, "r")
    # Split the input file in lines
    #ORIGINAL: raw_file_lines = raw_file.readlines(), leaves /r/n on last column entires
    
    raw_file_lines = raw_file.read().splitlines()
    for line in raw_file_lines:
        # Split each line on the column split symbol ";"
        raw_line_split = line.split(';')
        # Add the split row to the new file
        compiled_file_source_writer.writerow(raw_line_split)
    raw_file.close()
compiled_file_source.close()

In [27]:
# Pandas sanity check, open the just created file
pd.read_csv(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv")

,"﻿""Perioden""","""Regio's""","""Bevolking/Bevolkingssamenstelling op 1 januari/Totale bevolking (aantal)""","""Bevolking/Bevolkingssamenstelling op 1 januari/Geslacht/Mannen (aantal)""","""Bevolking/Bevolkingssamenstelling op 1 januari/Geslacht/Vrouwen (aantal)""","""Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/Jonger dan 5 jaar (aantal)""","""Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/5 tot 10 jaar (aantal)""","""Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/10 tot 15 jaar (aantal)""","""Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/15 tot 20 jaar (aantal)""","""Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/20 tot 25 jaar (aantal)""",...,"""Onderwijs/Naar woongemeente/Gediplomeerden/Wo master/doctoraal (aantal)""","""Arbeid/Banen van werknemers, relatief/A Landbouw, bosbouw en visserij (%)""","""Arbeid/Banen van werknemers, relatief/B-F Nijverheid en energie (%)""","""Arbeid/Banen van werknemers, relatief/G-N Commerciële dienstverlening (%)""","""Arbeid/Banen van werknemers, relatief/O-U Niet-commerciële dienstverlening (%)""","""Nabijheid voorzieningen/Gezondheid/Afstand tot huisartsenpraktijk (km)""","""Nabijheid voorzieningen/Gezondheid/Aantal huisartsenpraktijken binnen 3 km (aantal)""","""Nabijheid voorzieningen/Gezondheid/Afstand tot huisartsenpost (km)""","""Nabijheid voorzieningen/Gezondheid/Afstand tot ziekenhuis (km)""","""Nabijheid voorzieningen/Gezondheid/Aantal ziekenhuizen binnen 20 km (aantal)""\r\n"
0,"""2019""","""Aa en Hunze""",25386.0,12659.0,12727.0,922.0,1174.0,1366.0,1579.0,1031.0,...,8.0,1.0,10.0,45.0,44.0,"2,4","1,3","11,1","11,1","1,4\r\n"
1,"""2020""","""Aa en Hunze""",25445.0,12708.0,12737.0,946.0,1160.0,1354.0,1545.0,1079.0,...,NaN,1.0,8.0,43.0,47.0,"2,4","1,1","11,1","11,1","1,4\r\n"
2,"""2021""","""Aa en Hunze""",25399.0,12709.0,12690.0,937.0,1137.0,1309.0,1508.0,1103.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\r\n
3,"""2019""","""Aalburg""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\r\n
4,"""2020""","""Aalburg""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\r\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8422,"""\r\n",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8423,\r\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8424,"""""""2021"""""",""""""De Wolden"""""",24374,12197,12177,1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8425,"""\r\n",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Source 3 preprocessing

### Source 4 preprocessing

In [28]:
# Source file cleaning
source_number = 4
source_folder_path = f"data/Source_{source_number}"
# Open the source folder to take all CSV's from
csv_files_Source = glob.glob(os.path.join(f"{source_folder_path}", "*.csv"))

# Open the file to store the new CSV in
compiled_file_source = open(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv", "a")
# Start the CSV writer
compiled_file_source_writer = csv.writer(compiled_file_source)

# Go through all raw source files in the provided folder
for file_path in csv_files_Source:
    raw_file = open(file_path, "r")
    # Split the input file in lines
    raw_file_lines = raw_file.read().splitlines()
    for line in raw_file_lines:
        # Split each line on the column split symbol ";"
        raw_line_split = line.split(';')
        # Add the split row to the new file
        compiled_file_source_writer.writerow(raw_line_split)
    raw_file.close()
compiled_file_source.close()

In [29]:
# Pandas sanity check, open the just created file
pd.read_csv(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv", on_bad_lines='skip')

,"﻿""Status cijfer""","""Perioden""","""Regio's""","""Inwoners naar geslacht en leeftijd/Mannen/Totaal mannen (aantal)""","""Inwoners naar geslacht en leeftijd/Mannen/Jonger dan 20 jaar (aantal)""","""Inwoners naar geslacht en leeftijd/Mannen/20 tot 65 jaar (aantal)""","""Inwoners naar geslacht en leeftijd/Mannen/65 tot 75 jaar (aantal)""","""Inwoners naar geslacht en leeftijd/Mannen/75 tot 85 jaar (aantal)""","""Inwoners naar geslacht en leeftijd/Mannen/85 jaar of ouder (aantal)""","""Inwoners naar geslacht en leeftijd/Vrouwen/Totaal vrouwen (aantal)""","""Inwoners naar geslacht en leeftijd/Vrouwen/Jonger dan 20 jaar (aantal)""","""Inwoners naar geslacht en leeftijd/Vrouwen/20 tot 65 jaar (aantal)""","""Inwoners naar geslacht en leeftijd/Vrouwen/65 tot 75 jaar (aantal)""","""Inwoners naar geslacht en leeftijd/Vrouwen/75 tot 85 jaar (aantal)""","""Inwoners naar geslacht en leeftijd/Vrouwen/85 jaar of ouder (aantal)""","""Minderheden (aantal)""","""Banen van werknemers (x 1 000)""","""Provinciale wegen (km)""","""Inwoners in zorginstellingen (aantal)"""
0,"""Definitief""","""2019""","""Aa en Hunze""",12659.0,2623.0,6912.0,1952.0,941.0,231.0,12727.0,2418.0,6820.0,1959.0,1079.0,451.0,300.0,NaN,NaN,520.0
1,"""Definitief""","""2019""","""Aalburg""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"""Definitief""","""2019""","""Aalsmeer""",15694.0,3973.0,9031.0,1585.0,869.0,236.0,16034.0,3795.0,9099.0,1628.0,1035.0,477.0,1460.0,NaN,NaN,360.0
3,"""Definitief""","""2019""","""Aalten""",13559.0,3109.0,7686.0,1680.0,888.0,196.0,13452.0,2870.0,7382.0,1683.0,1044.0,473.0,570.0,NaN,NaN,460.0
4,"""Definitief""","""2019""","""Abcoude""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476,"""Definitief""","""2019""","""Zwartewaterland""",11449.0,3266.0,6348.0,1186.0,518.0,131.0,11054.0,3026.0,6047.0,1147.0,580.0,254.0,285.0,NaN,NaN,230.0
477,"""Definitief""","""2019""","""Zwijndrecht""",21656.0,5043.0,12156.0,2540.0,1451.0,466.0,22983.0,4776.0,12631.0,2825.0,1899.0,852.0,4465.0,NaN,NaN,390.0
478,"""Definitief""","""2019""","""Zwolle""",62794.0,15485.0,38381.0,5466.0,2674.0,788.0,64703.0,14943.0,38843.0,5931.0,3420.0,1566.0,7300.0,NaN,NaN,1830.0
479,"""Definitief""","""2019""","""Buitenland""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Source 5 preprocessing

In [10]:
# INFO ONLY ON PROVINCES

In [30]:
# Source file cleaning
source_number = 5
source_folder_path = f"data/Source_{source_number}"
# Open the source folder to take all CSV's from
csv_files_Source = glob.glob(os.path.join(f"{source_folder_path}", "*.csv"))

# Open the file to store the new CSV in
compiled_file_source = open(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv", "a")
# Start the CSV writer
compiled_file_source_writer = csv.writer(compiled_file_source)

# Go through all raw source files in the provided folder
for file_path in csv_files_Source:
    raw_file = open(file_path, "r")
    # Split the input file in lines
    raw_file_lines = raw_file.read().splitlines()
    for line in raw_file_lines:
        # Split each line on the column split symbol ";"
        raw_line_split = line.split(';')
        # Add the split row to the new file
        compiled_file_source_writer.writerow(raw_line_split)
    raw_file.close()
compiled_file_source.close()

In [31]:
# Pandas sanity check, open the just created file
pd.read_csv(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv", on_bad_lines='skip')

,"﻿""Leeftijd""","""Beroepen en specialismen""","""Sociaaleconomische categorie""","""Perioden""","""Regio's""","""Medisch geschoolden (aantal)"""
0,"""Totaal leeftijd""","""Totaal BIG-geregistreerde beroepen""","""Werknemer of zelfstandige""","""2019""","""Groningen (PV)""",13940
1,"""Totaal leeftijd""","""Totaal BIG-geregistreerde beroepen""","""Werknemer of zelfstandige""","""2019""","""Fryslân (PV)""",11740
2,"""Totaal leeftijd""","""Totaal BIG-geregistreerde beroepen""","""Werknemer of zelfstandige""","""2019""","""Drenthe (PV)""",10210
3,"""Totaal leeftijd""","""Totaal BIG-geregistreerde beroepen""","""Werknemer of zelfstandige""","""2019""","""Overijssel (PV)""",21755
4,"""Totaal leeftijd""","""Totaal BIG-geregistreerde beroepen""","""Werknemer of zelfstandige""","""2019""","""Flevoland (PV)""",5545
5,"""Totaal leeftijd""","""Totaal BIG-geregistreerde beroepen""","""Werknemer of zelfstandige""","""2019""","""Gelderland (PV)""",39925
6,"""Totaal leeftijd""","""Totaal BIG-geregistreerde beroepen""","""Werknemer of zelfstandige""","""2019""","""Utrecht (PV)""",28600
7,"""Totaal leeftijd""","""Totaal BIG-geregistreerde beroepen""","""Werknemer of zelfstandige""","""2019""","""Noord-Holland (PV)""",49195
8,"""Totaal leeftijd""","""Totaal BIG-geregistreerde beroepen""","""Werknemer of zelfstandige""","""2019""","""Zuid-Holland (PV)""",58480
9,"""Totaal leeftijd""","""Totaal BIG-geregistreerde beroepen""","""Werknemer of zelfstandige""","""2019""","""Zeeland (PV)""",6200


### Source 6 preprocessing

In [32]:
# Source file cleaning
source_number = 6
source_folder_path = f"data/Source_{source_number}"
# Open the source folder to take all CSV's from
csv_files_Source = glob.glob(os.path.join(f"{source_folder_path}", "*.csv"))

# Open the file to store the new CSV in
compiled_file_source = open(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv", "a")
# Start the CSV writer
compiled_file_source_writer = csv.writer(compiled_file_source)

# Go through all raw source files in the provided folder
for file_path in csv_files_Source:
    raw_file = open(file_path, "r")
    # Split the input file in lines
    raw_file_lines = raw_file.read().splitlines()
    for line in raw_file_lines:
        # Split each line on the column split symbol ";"
        raw_line_split = line.split(';')
        # Add the split row to the new file
        compiled_file_source_writer.writerow(raw_line_split)
    raw_file.close()
compiled_file_source.close()

In [33]:
# Pandas sanity check, open the just created file
pd.read_csv(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv")

,"﻿""Donorregistratie""","""Perioden""","""Regio's""","""Donorregister, vastgelegde keuze (x 1 000)"""
0,"""Totaal wel of niet geregistreerd""","""2019""","""Aa en Hunze""","22,8"
1,"""Totaal wel of niet geregistreerd""","""2020""","""Aa en Hunze""","22,8"
2,"""Totaal wel of niet geregistreerd""","""2021*""","""Aa en Hunze""","22,8"
3,"""Totaal wel of niet geregistreerd""","""2019""","""Aalsmeer""","27,3"
4,"""Totaal wel of niet geregistreerd""","""2020""","""Aalsmeer""","27,6"
...,...,...,...,...
3163,"""Niet geregistreerd in donorregister""","""2020""","""Zwijndrecht""","20,9"
3164,"""Niet geregistreerd in donorregister""","""2021*""","""Zwijndrecht""","3,2"
3165,"""Niet geregistreerd in donorregister""","""2019""","""Zwolle""","59,1"
3166,"""Niet geregistreerd in donorregister""","""2020""","""Zwolle""","56,2"


### Source 7 preprocessing

In [34]:
# Source file cleaning
source_number = 7
source_folder_path = f"data/Source_{source_number}"
# Open the source folder to take all CSV's from
csv_files_Source = glob.glob(os.path.join(f"{source_folder_path}", "*.csv"))

# Open the file to store the new CSV in
compiled_file_source = open(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv", "a")
# Start the CSV writer
compiled_file_source_writer = csv.writer(compiled_file_source)

# Go through all raw source files in the provided folder
for file_path in csv_files_Source:
    raw_file = open(file_path, "r")
    # Split the input file in lines
    raw_file_lines = raw_file.read().splitlines()
    for line in raw_file_lines:
        # Split each line on the column split symbol ";"
        raw_line_split = line.split(';')
        # Add the split row to the new file
        compiled_file_source_writer.writerow(raw_line_split)
    raw_file.close()
compiled_file_source.close()

In [35]:
# Pandas sanity check, open the just created file
pd.read_csv(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv")

,"﻿""Geslacht""","""Leeftijd""","""Geneesmiddelengroep (ATC)""","""Perioden""","""Regio's""","""Personen met geneesmiddelen, relatief (%)"""
0,"""Mannen""","""0 tot 15 jaar""","""Totaal""","""2019""","""Aa en Hunze""","44,33"
1,"""Mannen""","""0 tot 15 jaar""","""Totaal""","""2020*""","""Aa en Hunze""",39
2,"""Mannen""","""0 tot 15 jaar""","""Totaal""","""2019""","""Aalburg""",NaN
3,"""Mannen""","""0 tot 15 jaar""","""Totaal""","""2020*""","""Aalburg""",NaN
4,"""Mannen""","""0 tot 15 jaar""","""Totaal""","""2019""","""Aalsmeer""","48,55"
...,...,...,...,...,...,...
13659,"""Vrouwen""","""75 jaar of ouder""","""Totaal""","""2020*""","""Zwartewaterland""","88,11"
13660,"""Vrouwen""","""75 jaar of ouder""","""Totaal""","""2019""","""Zwijndrecht""","89,41"
13661,"""Vrouwen""","""75 jaar of ouder""","""Totaal""","""2020*""","""Zwijndrecht""","89,64"
13662,"""Vrouwen""","""75 jaar of ouder""","""Totaal""","""2019""","""Zwolle""","89,72"


### Source 9 preprocessing

In [36]:
# Source file cleaning
source_number = 9
source_folder_path = f"data/Source_{source_number}"
# Open the source folder to take all CSV's from
csv_files_Source = glob.glob(os.path.join(f"{source_folder_path}", "*.csv"))

# Open the file to store the new CSV in
compiled_file_source = open(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv", "a")
# Start the CSV writer
compiled_file_source_writer = csv.writer(compiled_file_source)

# Go through all raw source files in the provided folder
for file_path in csv_files_Source:
    raw_file = open(file_path, "r")
    # Split the input file in lines
    raw_file_lines = raw_file.read().splitlines()
    for line in raw_file_lines:
        # Split each line on the column split symbol ";"
        raw_line_split = line.split(';')
        # Add the split row to the new file
        compiled_file_source_writer.writerow(raw_line_split)
    raw_file.close()
compiled_file_source.close()

In [37]:
# Pandas sanity check, open the just created file
pd.read_csv(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv")

,"﻿""Geslacht""","""Leeftijd""","""Soort opname""","""Diagnose""","""Perioden""","""Regio's""","""Opnamen per 10 000 inwoners (per 10 000 inwoners)"""
0,"""Mannen""","""0 jaar""","""Totaal opnamen""","""Totaal alle diagnosen""","""2019*""","""Aa en Hunze""","6874,8"
1,"""Mannen""","""0 jaar""","""Totaal opnamen""","""Totaal alle diagnosen""","""2019*""","""Aalburg""",NaN
2,"""Mannen""","""0 jaar""","""Totaal opnamen""","""Totaal alle diagnosen""","""2019*""","""Aalsmeer""","7042,1"
3,"""Mannen""","""0 jaar""","""Totaal opnamen""","""Totaal alle diagnosen""","""2019*""","""Aalten""","7558,6"
4,"""Mannen""","""0 jaar""","""Totaal opnamen""","""Totaal alle diagnosen""","""2019*""","""Achtkarspelen""","9019,8"
...,...,...,...,...,...,...,...
5119,"""Vrouwen""","""80 jaar of ouder""","""Totaal opnamen""","""Totaal alle diagnosen""","""2019*""","""Zundert""","3212,7"
5120,"""Vrouwen""","""80 jaar of ouder""","""Totaal opnamen""","""Totaal alle diagnosen""","""2019*""","""Zutphen""","3625,6"
5121,"""Vrouwen""","""80 jaar of ouder""","""Totaal opnamen""","""Totaal alle diagnosen""","""2019*""","""Zwartewaterland""","3108,6"
5122,"""Vrouwen""","""80 jaar of ouder""","""Totaal opnamen""","""Totaal alle diagnosen""","""2019*""","""Zwijndrecht""","3811,3"


### Source 11 preprocessing

In [38]:
# Source file cleaning
source_number = 11
source_folder_path = f"data/Source_{source_number}"
# Open the source folder to take all CSV's from
csv_files_Source = glob.glob(os.path.join(f"{source_folder_path}", "*.csv"))

# Open the file to store the new CSV in
compiled_file_source = open(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv", "a")
# Start the CSV writer
compiled_file_source_writer = csv.writer(compiled_file_source)

# Go through all raw source files in the provided folder
for file_path in csv_files_Source:
    raw_file = open(file_path, "r")
    # Split the input file in lines
    raw_file_lines = raw_file.read().splitlines()
    for line in raw_file_lines:
        # Split each line on the column split symbol ";"
        raw_line_split = line.split(';')
        # Add the split row to the new file
        compiled_file_source_writer.writerow(raw_line_split)
    raw_file.close()
compiled_file_source.close()

In [39]:
# Pandas sanity check, open the just created file
pd.read_csv(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv")

,"﻿""Perioden""","""Regio's""","""Totaal alle onderliggende doodsoorzaken (aantal)""","""Nieuwvormingen (aantal)""","""Ziekten van hart en vaatstelsel (aantal)""","""Ziekten van ademhalingsstelsel (aantal)"""
0,"""2018""","""Aa en Hunze""",283.0,96.0,79.0,14.0
1,"""2019""","""Aa en Hunze""",280.0,94.0,86.0,18.0
2,"""2020""","""Aa en Hunze""",274.0,88.0,77.0,14.0
3,"""2018""","""Aalburg""",84.0,27.0,23.0,8.0
4,"""2019""","""Aalburg""",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2131,"""2019""","""Zwijndrecht""",471.0,154.0,121.0,32.0
2132,"""2020""","""Zwijndrecht""",558.0,142.0,123.0,32.0
2133,"""2018""","""Zwolle""",988.0,287.0,266.0,74.0
2134,"""2019""","""Zwolle""",953.0,294.0,258.0,65.0


### Source 14 preprocessing

In [40]:
# Source file cleaning
source_number = 14
source_folder_path = f"data/Source_{source_number}"
# Open the source folder to take all CSV's from
csv_files_Source = glob.glob(os.path.join(f"{source_folder_path}", "*.csv"))

# Open the file to store the new CSV in
compiled_file_source = open(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv", "a")
# Start the CSV writer
compiled_file_source_writer = csv.writer(compiled_file_source)

# Go through all raw source files in the provided folder
for file_path in csv_files_Source:
    raw_file = open(file_path, "r")
    # Split the input file in lines
    raw_file_lines = raw_file.read().splitlines()
    for line in raw_file_lines:
        # Split each line on the column split symbol ";"
        raw_line_split = line.split(';')
        # Add the split row to the new file
        compiled_file_source_writer.writerow(raw_line_split)
    raw_file.close()
compiled_file_source.close()

In [41]:
# Pandas sanity check, open the just created file
pd.read_csv(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv")

,"﻿""Perioden""","""Regio's""","""Bevolking aan het begin van de periode (aantal)""","""Levend geboren kinderen (aantal)""","""Overledenen (aantal)""","""Vertrek uit de gemeente/Vertrek naar andere gemeente (aantal)""","""Bevolkingsgroei/Bevolkingsgroei, relatief (%)"""
0,"""2019""","""Aa en Hunze""",25386.0,176.0,280.0,1305.0,"0,23"
1,"""2020""","""Aa en Hunze""",25445.0,167.0,274.0,1395.0,"-0,18"
2,"""2021*""","""Aa en Hunze""",25399.0,198.0,284.0,1182.0,"0,69"
3,"""2019""","""Aalburg""",NaN,NaN,NaN,NaN,NaN
4,"""2020""","""Aalburg""",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1660,"""2020""","""Zwijndrecht""",44737.0,406.0,558.0,1969.0,"0,08"
1661,"""2021*""","""Zwijndrecht""",44775.0,490.0,535.0,2289.0,"-0,01"
1662,"""2019""","""Zwolle""",127497.0,1460.0,953.0,5356.0,"1,05"
1663,"""2020""","""Zwolle""",128840.0,1469.0,1036.0,5574.0,"0,78"
